In [1]:
import os 
import urllib
from os import path
import json
import requests
import time
from io import BytesIO
from PIL import Image, ImageOps
import base64

In [2]:
# Check that docker is working
!docker run --rm hello-world

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world

Digest: sha256:c5515758d4c5e1e838e9cd307f6c6a0d620b5e07e6f927b07d05f6d12a1ac8d7
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://cloud.docker.com/

For more examples and ideas, visit:
 https://docs.do

### Step 1. Create WebApp Code

In [3]:
%%bash
mkdir script
mkdir script/code

mkdir: cannot create directory ‘script’: File exists
mkdir: cannot create directory ‘script/code’: File exists


In [32]:
%%writefile script/code/model.py

import base64
import urllib
import numpy as np
import cntk
import pkg_resources
from flask import Flask, request
import json
from io import BytesIO
from PIL import Image, ImageOps
from cntk import load_model, combine

app = Flask(__name__)
print("Something outside of @app.route() is always loaded")

# Pre-load model
MODEL = load_model("ResNet_18.model")
print("Loaded model: ", MODEL)
# Pre-load labels
with open('synset-1k.txt', 'r') as f:
    LABELS = [l.rstrip() for l in f]
print(LABELS[:10])
print("Loaded {0} labels".format(len(LABELS)))

@app.route("/")
def healthy_me():
    return "healthy"

@app.route('/cntk')
def cntk_ver():
    return "CNTK version: {}".format(pkg_resources.get_distribution("cntk").version)

@app.route('/posttest', methods=['POST'])
def posttest():
    return "POST healthy"

@app.route('/posttest_input', methods=['POST'])
def posttest_input():
    return json.dumps(request.json['input'])

@app.route("/api/uploader", methods=['POST'])
def api_upload_file():
    inputString = request.json['input']
    images = json.loads(inputString)   
    for base64ImgString in images:
        if base64ImgString.startswith('b\''):
            base64ImgString = base64ImgString[2:-1]
        base64Img = base64ImgString.encode('utf-8')
        # Preprocess the input data 
        decoded_img = base64.b64decode(base64Img)
        img_buffer = BytesIO(decoded_img)
        # Load image with PIL (RGB)
        img = Image.open(img_buffer).convert('RGB')
        img = ImageOps.fit(img, (224, 224), Image.ANTIALIAS)
        return json.dumps(run_some_deep_learning_cntk(img))

def run_some_deep_learning_cntk(rgb_pil_image):
    # Convert to BGR
    rgb_image = np.array(rgb_pil_image, dtype=np.float32)
    bgr_image = rgb_image[..., [2, 1, 0]]
    img = np.ascontiguousarray(np.rollaxis(bgr_image, 2))

    # Use last layer to make prediction
    z_out = combine([MODEL.outputs[3].owner])
    result = np.squeeze(z_out.eval({z_out.arguments[0]: [img]}))

    # Sort probabilities 
    a = np.argsort(result)[-1]
    predicted_category = " ".join(LABELS[a].split(" ")[1:])
    return predicted_category

if __name__ == '__main__':
    # This is just for debugging
    app.run(host='0.0.0.0', port=5005)

Overwriting script/code/model.py


In [5]:
%%writefile script/code/flaskconfig
server {
    listen 80;
    location = /favicon.ico { access_log off; log_not_found off; }
    location / {
        proxy_set_header Host $http_host;
        proxy_set_header X-Real-IP $remote_addr;
        proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
        proxy_set_header X-Forwarded-Proto $scheme;
        proxy_pass http://0.0.0.0:5005;
    }
}

Overwriting script/code/flaskconfig


In [6]:
%%writefile script/code/start.sh
service nginx start
/usr/local/bin/gunicorn --workers 1 -m 007 --bind 0.0.0.0:5005 model:app

Overwriting script/code/start.sh


In [7]:
%%writefile script/code/requirements.txt
Flask
gunicorn
pillow

Overwriting script/code/requirements.txt


In [8]:
urllib.urlretrieve("https://bootstrap.pypa.io/get-pip.py", "script/code/get-pip.py")

('script/code/get-pip.py', <httplib.HTTPMessage instance at 0x7efdc80f7c68>)

In [9]:
urllib.urlretrieve('https://azurewebappcntk.blob.core.windows.net/model/ResNet_18.model', 'script/code/ResNet_18.model')

('script/code/ResNet_18.model',
 <httplib.HTTPMessage instance at 0x7efdc80f7638>)

In [10]:
urllib.urlretrieve('https://azurewebappcntk.blob.core.windows.net/model/synset-1k.txt', 'script/code/synset-1k.txt')

('script/code/synset-1k.txt', <httplib.HTTPMessage instance at 0x7efdc810f368>)

### Step 2. LogIn

In [ ]:
!az login -o table
docker_registry = "ikmscontainer"
docker_registry_group = "ikmscontainergorup"
!az group create -n $docker_registry_group -l southcentralus -o table
!az acr create -n $docker_registry -g $docker_registry_group -l southcentralus -o table
!az acr update -n $docker_registry --admin-enabled true -o table
json_data = !az acr credential show -n $docker_registry
docker_username = json.loads(''.join(json_data))['username']
docker_password = json.loads(''.join(json_data))['password']
print(docker_username)
print(docker_password)
json_data = !az acr show -n $docker_registry
docker_registry_server = json.loads(''.join(json_data))['loginServer']

### Step 3. Create Docker Image

In [12]:
!mkdir script/docker

mkdir: cannot create directory ‘script/docker’: File exists


In [34]:
%%writefile script/docker/dockerfile

FROM ubuntu:16.04
RUN mkdir /code
WORKDIR /code
MAINTAINER Ilia Karmanov
ADD code /code
RUN apt-get update && apt-get install -y --no-install-recommends \
        openmpi-bin \
        python3 \
        python3-dev \
        python3-setuptools \
        curl \
        nginx &&\
        python3 /code/get-pip.py && \
        rm /etc/nginx/sites-enabled/default && \
        cp /code/flaskconfig /etc/nginx/sites-available/ && \
        ln -s /etc/nginx/sites-available/flaskconfig /etc/nginx/sites-enabled/ && \
        python3 -m pip install https://cntk.ai/PythonWheel/CPU-Only/cntk-2.0.beta15.0-cp35-cp35m-linux_x86_64.whl && \
        python3 -m pip install -r /code/requirements.txt && \
        chmod 777 /code/start.sh
EXPOSE 80
ENTRYPOINT /code/start.sh

Overwriting script/docker/dockerfile


In [35]:
container_name = docker_registry_server + "/ilkarman/dkili"
application_path = 'script'
docker_file_location = path.join(application_path, 'docker/dockerfile')

In [36]:
!docker login $docker_registry_server -u $docker_username -p $docker_password

Login Succeeded


In [37]:
container_name

u'ikmscontainer.azurecr.io/ilkarman/dkili'

In [ ]:
%%bash
docker stop $(docker ps -a -q)
docker rm $(docker ps -a -q)

In [39]:
# Running from shell:
docker_build = "docker build -t {0} -f {1} {2} --no-cache".format(container_name, docker_file_location, application_path)
docker_build

'docker build -t ikmscontainer.azurecr.io/ilkarman/dkili -f script/docker/dockerfile script --no-cache'

In [ ]:
# This will take a while; potentially run from shell instead to see output (there will be a lot)
!$docker_build

Test everything is working locally before pushing

In [41]:
# 1.23GB (ResNet_18.model is ~60MB)
#!docker images   

In [42]:
# To debug
print(container_name)
# In shell (run interactive mode):
#docker run -p 8070:8090 -it $container_name
#conda info --env
#which python
# ... etc

ikmscontainer.azurecr.io/ilkarman/dkili


In [43]:
test_cont = !docker run -p 80:80 -d $container_name
test_cont

['526cc1a8b9a35f3b2f3704f7e542fe7dbb2f5a42aff7ab76c3708776fb6f087e']

In [44]:
time.sleep(5)  # Wait to load

In [45]:
!curl http://0.0.0.0

healthy

In [46]:
!curl http://0.0.0.0/cntk

CNTK version: 2.0.beta15.0

In [47]:
IMAGEURL = "https://www.britishairways.com/assets/images/information/about-ba/fleet-facts/airbus-380-800/photo-gallery/240x295-BA-A380-exterior-2-high-res.jpg"

In [76]:
def url_img_to_json_img(url):
    bytfile = BytesIO(urllib.urlopen(url).read())
    img = Image.open(bytfile).convert('RGB')  # 3 Channels
    img = ImageOps.fit(img, (40, 40), Image.ANTIALIAS)  # Fixed size 
    imgio = BytesIO()
    img.save(imgio, 'PNG')
    imgio.seek(0)
    dataimg = base64.b64encode(imgio.read())
    return json.dumps(
        {'input':'[\"{0}\"]'.format(dataimg.decode('utf-8'))})

In [77]:
jsonimg = url_img_to_json_img(IMAGEURL)
jsonimg

'{"input": "[\\"iVBORw0KGgoAAAANSUhEUgAAACgAAAAoCAIAAAADnC86AAANEklEQVR4nFVXW8wkV3GuOuf06cv0zPwz/3X+va93ba+1XtvxFQgOiiNZVoIiJJK3RELkORApD6AoD8ljFCmIPOUJIUWISJEQhBCDMQGCcRzAF2xvbK/t3d97++9z657uPudUVR5m/mXTUkul7qr6LlVnLvjs37wgAvNLgAFQAODoEQIAoIDMI1nci3QRhYgAcCdhXoGgBOCuXFSIRyEgIiIagyC4SBFAEZGjclwUigAsAEQAYZ4COE+To35zcAEEPGJxVI7z8qMbENFoxDtK5C6OdzLvvo5aowAIyF0ACHMBCzqycA4WnBGPuIICwDmwLMxc+KsAREAQ8S7gBYYA32EmoBBAFi1BwQIYBeWOesAjYJS7YgBlzNwNlKOp3K157oQILEYkCEeDEUAQxDsvEVDmgcybIAAKggKU3xgnCIgAqNgsEEUQABUCCstc7tGsFx7MGYDgb1ZCgbrTFP8fYxQ8kjsfjSAgaIUsMBdt7khEUMXMoYIsjphEzb2Z7xIKgCxmoBhFAYAg4pwJKpjLRTlSPDdajnZZrEEWqD0m0YKeQlAgiICNp+eeGDx270pROo3IwsLALAwsIiDADCICPMcHYFkEIiCALCiiRJSIRlAAGkAJaEANGII892D74QELi0bQKAoBFKJWgkbr/e0vPBGdOrFUVl4rtTBOEARZFmuEoOZ6EATnA1pgC7IAswjxnRPCjMK1owcGyaYetZVjBgWCAgpEMiW5xpaGH1+jV//5u59/sqPzPHiOFbaNeAEEsQoAIEIAAGJhQQEkABIMPGeEi4kBogiwCBMzTWbNo6eypwbVzcP6l7eMQWACFlEsXHhCwFyxand/Thv0T//w6d6e0zYzohEzJe1IZRqBUQFq4E6ELY29WCUKUy25BYsiQg2BY/AMgYVBEgWxyMfOLT2y3hRF8cauaQIoIRBB

In [49]:
jsonimg[:100]  # Example of json string
headers = {'content-type': 'application/json'}

requests.post('http://0.0.0.0/api/uploader', 
              data=jsonimg,
              headers=headers).json()

u'airliner'

In [50]:
requests.post(
    'http://0.0.0.0/posttest_input', 
    data=json.dumps({'input':'[\"{0}\"]'.format('ilia')}),
    headers=headers).json()

u'["ilia"]'

In [51]:
!docker kill {test_cont[0]}

526cc1a8b9a35f3b2f3704f7e542fe7dbb2f5a42aff7ab76c3708776fb6f087e


### Step 4. Push Docker Image to Registry

In [52]:
container_name

u'ikmscontainer.azurecr.io/ilkarman/dkili'

In [53]:
!docker push $container_name

The push refers to a repository [ikmscontainer.azurecr.io/ilkarman/dkili]

91cafdd6: Preparing 
07099e84: Preparing 
651eed25: Preparing 
7159aa8b: Preparing 
02c3dcde: Preparing 
0d0e5bb2: Preparing 
2553e37a: Preparing 
91cafdd6: Pushed  1.109 GB/1.102 GBpp latest: digest: sha256:2a1a4dda08e77895f3790752cfe4e8a6d0428f8818a3dfa869bdcbb601a1ae5d size: 1988


In [54]:
!curl http://mailiahack.azurewebsites.net/

healthy

In [55]:
!curl http://mailiahack.azurewebsites.net/cntk

CNTK version: 2.0.beta15.0

In [82]:
jsonimg = url_img_to_json_img(IMAGEURL)
jsonimg[:100]  # Example of json strin

'{"input": "[\\"iVBORw0KGgoAAAANSUhEUgAAACgAAAAoCAIAAAADnC86AAANEklEQVR4nFVXW8wkV3GuOuf06cv0zPwz/3X+va'

In [ ]:
# Public docker hub: masalvar/webcntkgunicorn

In [83]:
headers = {'content-type': 'application/json'}
requests.post('http://mscntkacsagents.southcentralus.cloudapp.azure.com/score', 
              data=jsonimg,
              headers=headers).json()

{u'result': [[[[u'n04266014 space shuttle', 1105.2443504333496],
    [u'n02641379 gar, garfish, garpike, billfish, Lepisosteus osseus',
     1070.7453727722168],
    [u'n02690373 airliner', 1020.3579902648926]]],
  u'Computed in 430.18 ms']}

In [74]:
headers = {'content-type': 'application/json'}
requests.post('http://webappik.azurewebsites.net/score', 
              data=jsonimg,
              headers=headers).content

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>500 Internal Server Error</title>\n<h1>Internal Server Error</h1>\n<p>The server encountered an internal error and was unable to complete your request.  Either the server is overloaded or there is an error in the application.</p>\n'

In [64]:
requests.post(
    'http://testdockik.azurewebsites.net/posttest_input', 
    data=json.dumps({'input':'[\"{0}\"]'.format('ilia')}),
    headers=headers).content

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server.  If you entered the URL manually please check your spelling and try again.</p>\n'